In [2]:
import cv2
import mediapipe as mp

2023-06-01 17:28:15.821438: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 17:28:16.472690: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guillaume/anaconda3/envs/semproj/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda-11.7/lib64:
2023-06-01 17:28:16.472751: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guillaume/ana

In [3]:
def get_bounding_box(hand_landmarks, image_width, image_height):
    x_min = image_width
    y_min = image_height
    x_max = 0
    y_max = 0

    for landmark in hand_landmarks.landmark:
        x = int(landmark.x * image_width)
        y = int(landmark.y * image_height)

        if x < x_min:
            x_min = x
        if y < y_min:
            y_min = y
        if x > x_max:
            x_max = x
        if y > y_max:
            y_max = y

    # Add some padding to the bounding box
    padding = 20
    x_min -= padding
    y_min -= padding
    x_max += padding
    y_max += padding

    # Ensure the bounding box coordinates are within the image boundaries
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(image_width, x_max)
    y_max = min(image_height, y_max)

    return x_min, y_min, x_max, y_max

In [4]:

# Open the webcam
cap = cv2.VideoCapture(0)  # 0 represents the default webcam, change it if necessary

# Check if the webcam is successfully opened
if not cap.isOpened():
    print("Unable to open the webcam.")
    exit()

# Initialize MediaPipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Counter for captured frames
frame_count = 0

n_frame = 0
while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Check if the frame is successfully read
    if not ret:
        print("Failed to read the frame.")
        break

    n_frame += 1
    # Flip the frame horizontally for a selfie view
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands in the frame
    results = hands.process(frame_rgb)

    # Check if hands are detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Extract the bounding box coordinates for the hand
            x_min, y_min, x_max, y_max = get_bounding_box(hand_landmarks, frame.shape[1], frame.shape[0])

            # Crop the image around the hand region
            hand_image = frame[y_min:y_max, x_min:x_max]

            # Save the cropped hand image
            cv2.imwrite(f"hands/hands_down/1_{frame_count}.png", hand_image)

            # Increment the frame count
            frame_count += 1

    # Display the frame in a window named 'Webcam Feed'
    cv2.imshow('Webcam Feed', frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) == 27:  # 27 is the ASCII code for the Escape key
        break

    if n_frame > 300:
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()






INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
